In [1]:
# install metacells
#pip install metacells

# import libraries for visualization
import anndata as ad
import matplotlib.pyplot as plt
import metacells as mc
import numpy as np
import os
import pandas as pd
import scipy.sparse as sp
import seaborn as sb
import scanpy

from math import hypot
from matplotlib.collections import LineCollection
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('svg')
sb.set_style("white")

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`


In [ ]:
!ls "../../Alyssa/Single_Cell_CHIP/demultiplexed/DNMT3A_Mutant"
!cd ../../Alyssa/Single_Cell_CHIP
#!mkdir h5ad_files

In [ ]:
tet2_stim = scanpy.read_10x_mtx("../../Alyssa/Single_Cell_CHIP/demultiplexed/TET2 STIM")
tet2_stim.write_h5ad("../../Alyssa/Single_Cell_CHIP/h5ad_files/tet2_stim.h5ad")

In [ ]:
!head ../../Alyssa/Single_Cell_CHIP/metacells/no_veh_metacells.csv

In [ ]:
!ls "../../Alyssa/Single_Cell_CHIP/h5ad_files/"

In [ ]:
!gsutil cp -r ../../Alyssa/Single_Cell_CHIP/h5ad_files gs://fc-112f611a-aca5-42eb-9970-5050086b3e8d/Single_Cell_CHIP/ 

In [ ]:
raw = ad.read_h5ad("../../Alyssa/Single_Cell_CHIP/h5ad_files/tet2_stim.h5ad")
mc.ut.set_name(raw, "PBMC")
print(raw.shape)

In [ ]:
excluded_gene_names = ['IGHMBP2', 'IGLL1', 'IGLL5', 'IGLON5', 'NEAT1', 'TMSB10', 'TMSB4X']
excluded_gene_patterns = ['MT-.*']

In [ ]:
# exclude low quality genes
mc.pl.analyze_clean_genes(raw,
                          excluded_gene_names=excluded_gene_names,
                          excluded_gene_patterns=excluded_gene_patterns,
                          random_seed=123456)

In [ ]:
mc.pl.pick_clean_genes(raw)
raw.write("../../Alyssa/Single_Cell_CHIP/h5ad_files/full_tet2_stim.h5ad")
full = raw

In [ ]:
full = ad.read_h5ad("../../Alyssa/Single_Cell_CHIP/h5ad_files/full_tet2_stim.h5ad")

In [ ]:
properly_sampled_max_excluded_genes_fraction = 0.1
properly_sampled_min_cell_total = 800
properly_sampled_max_cell_total = 8000

mc.pl.analyze_clean_cells(
    full,
    properly_sampled_min_cell_total=properly_sampled_min_cell_total,
    properly_sampled_max_cell_total=properly_sampled_max_cell_total,
    properly_sampled_max_excluded_genes_fraction=properly_sampled_max_excluded_genes_fraction)

In [ ]:
mc.pl.pick_clean_cells(full)

In [ ]:
clean = mc.pl.extract_clean_data(full)

In [ ]:
suspect_gene_names = ['PCNA', 'MKI67', 'TOP2A', 'HIST1H1D',
                      'FOS', 'JUN', 'HSP90AB1', 'HSPA1A',
                      'ISG15', 'WARS' ]
suspect_gene_patterns = [ 'MCM[0-9]', 'SMC[0-9]', 'IFI.*' ]
suspect_genes_mask = mc.tl.find_named_genes(clean, names=suspect_gene_names,
                                            patterns=suspect_gene_patterns)
suspect_gene_names = sorted(clean.var_names[suspect_genes_mask])

In [ ]:
mc.pl.relate_genes(clean, random_seed=123456)

In [ ]:
module_of_genes = clean.var['related_genes_module']
suspect_gene_modules = np.unique(module_of_genes[suspect_genes_mask])
suspect_gene_modules = suspect_gene_modules[suspect_gene_modules >= 0]
print(suspect_gene_modules)

In [ ]:
similarity_of_genes = mc.ut.get_vv_frame(clean, 'related_genes_similarity')
for gene_module in suspect_gene_modules:
    module_genes_mask = module_of_genes == gene_module
    similarity_of_module = similarity_of_genes.loc[module_genes_mask, module_genes_mask]
    similarity_of_module.index = \
    similarity_of_module.columns = [
        '(*) ' + name if name in suspect_gene_names else name
        for name in similarity_of_module.index
    ]
    ax = plt.axes()
    sb.heatmap(similarity_of_module, vmin=0, vmax=1, xticklabels=True, yticklabels=True, ax=ax, cmap="YlGnBu")
    ax.set_title(f'Gene Module {gene_module}')
    plt.show()
    print(similarity_of_module.mean().mean())

In [ ]:
forbidden_genes_mask = suspect_genes_mask
for gene_module in [28, 31, 44]:
    module_genes_mask = module_of_genes == gene_module
    forbidden_genes_mask |= module_genes_mask
forbidden_gene_names = sorted(clean.var_names[forbidden_genes_mask])
print(len(forbidden_gene_names))
print(' '.join(forbidden_gene_names))

In [ ]:
max_parallel_piles = mc.pl.guess_max_parallel_piles(clean)
print(max_parallel_piles)
mc.pl.set_max_parallel_piles(max_parallel_piles)

In [ ]:

with mc.ut.progress_bar():
    mc.pl.divide_and_conquer_pipeline(clean,
                                      forbidden_gene_names=forbidden_gene_names,
                                      #target_metacell_size=...,
                                      random_seed=123456)

In [ ]:
clean.obs["metacell"].to_csv("../../Alyssa/Single_Cell_CHIP/metacells/test.csv")


In [ ]:
metacells = mc.pl.collect_metacells(clean, name='PBMC.metacells')

In [ ]:
mc.pl.compute_umap_by_features(metacells, max_top_feature_genes=1000,
                               min_dist=2.0, random_seed=123456)

In [ ]:
umap_x = mc.ut.get_o_numpy(metacells, 'umap_x')
umap_y = mc.ut.get_o_numpy(metacells, 'umap_y')
plot = sb.scatterplot(x=umap_x, y=umap_y)

In [ ]:
name = metacells.uns['__name__']
del metacells.uns['__name__']
metacells.write('../../Alyssa/Single_Cell_CHIP/h5ad_files/tet2_stim_for_seurat.h5ad')

In [2]:
def run_metacells(group_name, target_metacell_size=160000):
    group_data = scanpy.read_10x_mtx("../../Alyssa/Single_Cell_CHIP/demultiplexed/" + group_name)
    h5ad_filename = "../../Alyssa/Single_Cell_CHIP/h5ad_files/" + group_name.lower().replace(" ", "_") + ".h5ad"
    group_data.write_h5ad(h5ad_filename)
    
    raw = ad.read_h5ad(h5ad_filename)
    mc.ut.set_name(raw, "PBMC")
    
    excluded_gene_names = ['IGHMBP2', 'IGLL1', 'IGLL5', 'IGLON5', 'NEAT1', 'TMSB10', 'TMSB4X']
    excluded_gene_patterns = ['MT-.*']
    
    # exclude low quality genes
    mc.pl.analyze_clean_genes(raw,
                          excluded_gene_names=excluded_gene_names,
                          excluded_gene_patterns=excluded_gene_patterns,
                          random_seed=123456)
    
    mc.pl.pick_clean_genes(raw)
    raw.write("../../Alyssa/Single_Cell_CHIP/h5ad_files/full_" + group_name.lower().replace(" ", "_") + ".h5ad")
    full = raw
    
    properly_sampled_min_cell_total = 800
    properly_sampled_max_cell_total = 8000
    properly_sampled_max_excluded_genes_fraction = 0.1
    
    mc.pl.analyze_clean_cells(
        full,
        properly_sampled_min_cell_total=properly_sampled_min_cell_total,
        properly_sampled_max_cell_total=properly_sampled_max_cell_total,
        properly_sampled_max_excluded_genes_fraction=properly_sampled_max_excluded_genes_fraction)
    
    mc.pl.pick_clean_cells(full)
    clean = mc.pl.extract_clean_data(full)
    
    suspect_gene_names = ['PCNA', 'MKI67', 'TOP2A', 'HIST1H1D',
                      'FOS', 'JUN', 'HSP90AB1', 'HSPA1A',
                      'ISG15', 'WARS' ]
    suspect_gene_patterns = [ 'MCM[0-9]', 'SMC[0-9]', 'IFI.*' ]
    suspect_genes_mask = mc.tl.find_named_genes(clean, names=suspect_gene_names,
                                            patterns=suspect_gene_patterns)
    suspect_gene_names = sorted(clean.var_names[suspect_genes_mask])
    
    mc.pl.relate_genes(clean, random_seed=123456)
    
    module_of_genes = clean.var['related_genes_module']
    suspect_gene_modules = np.unique(module_of_genes[suspect_genes_mask])
    suspect_gene_modules = suspect_gene_modules[suspect_gene_modules >= 0]
    
    similarity_of_genes = mc.ut.get_vv_frame(clean, 'related_genes_similarity')
    suspect_modules = []
    
    #replace the for loop below with a condition (eg. if median similarity score for matrix is greater than 0.75, exclude module)
    for gene_module in suspect_gene_modules:
        module_genes_mask = module_of_genes == gene_module
        similarity_of_module = similarity_of_genes.loc[module_genes_mask, module_genes_mask]
        similarity_of_module.index = \
        similarity_of_module.columns = [
            '(*) ' + name if name in suspect_gene_names else name
            for name in similarity_of_module.index
        ]
        module_mean_value = similarity_of_module.mean().mean()
        if module_mean_value > 0.75:
            suspect_modules.append(gene_module)
            
    forbidden_genes_mask = suspect_genes_mask
    for gene_module in suspect_modules:
        module_genes_mask = module_of_genes == gene_module
        forbidden_genes_mask |= module_genes_mask
    forbidden_gene_names = sorted(clean.var_names[forbidden_genes_mask])
    
    max_parallel_piles = mc.pl.guess_max_parallel_piles(clean)
    mc.pl.set_max_parallel_piles(max_parallel_piles)
    mc.pl.set_max_parallel_piles(1)
    
    
    with mc.ut.progress_bar():
        mc.pl.divide_and_conquer_pipeline(clean,
                                          forbidden_gene_names=forbidden_gene_names,
                                          random_seed=123456,
                                          target_metacell_size=target_metacell_size)

    clean.obs["metacell"].to_csv("../../Alyssa/Single_Cell_CHIP/metacells/" + group_name.lower().replace(" ", "_") + "_metacells.csv")

    metacells = mc.pl.collect_metacells(clean, name='PBMC.metacells')
    #mc.pl.compute_umap_by_features(metacells, max_top_feature_genes=1000,
    #                           min_dist=2.0, random_seed=123456)
    
    name = metacells.uns['__name__']
    del metacells.uns['__name__']
    metacells.write("../../Alyssa/Single_Cell_CHIP/h5ad_files/" + group_name.lower().replace(" ", "_") + "_for_seurat.h5ad")
    

In [ ]:
#group_list = ["TET2 STIM", "TET2 VEH", "DNMT3A STIM", "DNMT3A VEH", "none STIM", "none VEH"]
#for group in group_list:
#    run_metacells(group)
    
run_metacells("DNMT3A STIM")

In [3]:
#run_metacells("TET2_Wildtype_CD14_Monos", 80000)
#run_metacells("TET2_Mutant_CD14_Monos", 80000)
#run_metacells("DNMT3A_Wildtype_CD14_Monos", 80000)
run_metacells("DNMT3A_Mutant_CD14_Monos", 80000)

set PBMC.var[properly_sampled_gene]: 12776 true (35.02%) out of 36485 bools
set PBMC.var[excluded_gene]: 6 true (0.01645%) out of 36485 bools
set PBMC.var[noisy_lonely_gene]: 0 true (0%) out of 36485 bools
set PBMC.var[clean_gene]: 12772 true (35.01%) out of 36485 bools
set PBMC.obs[properly_sampled_cell]: 308 true (100%) out of 308 bools
set PBMC.obs[clean_cell]: 308 true (100%) out of 308 bools
set PBMC.clean.obs[full_cell_index]: 308 int64s
set PBMC.clean.var[full_gene_index]: 12772 int64s
set PBMC.clean.var[related_genes_module]: 12151 outliers (95.14%) out of 12772 int32 elements with 36 groups with mean size 17.25
set PBMC.clean.varp[related_genes_similarity]: csr_matrix 12772 X 12772 float32s (385641 > 0)
Compute metacells for rare gene modules...
Compute common metacells...
